# Create Regression Models to Predict Income
## **A Notebook for Using Demographic Data to Predict Income/Wage**

#### Import the data from pickles, and separate into the features/targets

In [1]:
import pandas as pd
import numpy as np

In [2]:
from datetime import datetime

def log(message):
    print(datetime.now().strftime("%H:%M:%S -"), message)
    
def printnow():
    print(datetime.now().strftime("Current time: %H:%M:%S"))

In [3]:
log('Reading data')
us_personal = pd.read_pickle('preprocessed_data/onehot_data.zip')
log('Done\n')

22:42:00 - Reading data
22:42:16 - Done



In [4]:
from sklearn.preprocessing import RobustScaler # a scaler to overcome outliers from the data
from sklearn.linear_model import ElasticNet, SGDRegressor # baseline regression models
from keras import Sequential # all relevant imports for Keras (TensorFlow based neural net)
from keras.layers import Dense, LeakyReLU, ELU
from keras.wrappers.scikit_learn import KerasRegressor

# metrics to exaluate the performance of our regressors
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

Using TensorFlow backend.


In [5]:
# Separate the data into three groups: incomes or wages (targets), and features (all other columns)
# and return a corresponding dictionary with the three sets
def features_targets(df):
    features = df.drop(columns=['PINCP', 'WAGP'])
    incomes = df['PINCP']
    wages = df['WAGP']
    return {'features':features, 'incomes':incomes, 'wages':wages}

# create a dictionary representing the three sets and their features/targets
log('Splitting Train/Test Data')
train, test = train_test_split(us_personal, test_size=0.2)
dataset = {
    'train':features_targets(train),
    'test':features_targets(test)}


# Initialize the RobustScaler on the training data (fit before training each regression model)
log('Scaling Train/Test Features')
robust_scaler = RobustScaler().fit(dataset['train']['features'])
dataset['train']['features'] = robust_scaler.transform(dataset['train']['features'])
dataset['test']['features'] = robust_scaler.transform(dataset['test']['features'])
log('Done\n')

22:42:19 - Splitting Train/Test Data
22:42:40 - Scaling Train/Test Features
22:43:06 - Done



In [6]:
# a function to fit, test, and print results of a given estimator for a given target column
def fit_test_print(estimator, estimator_name, target, target_name, grid=1):
    log(f'Training {estimator_name}')
    estimator.fit(dataset['train']['features'], dataset['train'][target].array)
    y_pred_train = estimator.predict(dataset['train']['features'])
    y_pred_test = estimator.predict(dataset['test']['features'])
    log('Done\n')
    
    print(f"Results for {target_name} with {estimator_name}")
    if grid == 1: print(f"- Best parameters: {estimator.best_params_}")

    print(f"- Training Set")
    print(f"\tMean Squared Error: {mean_squared_error(dataset['train'][target], y_pred_train)}")
    print(f"\tMedian Absolute Error: {median_absolute_error(dataset['train'][target], y_pred_train)}")
    print(f"\tr-Squared: {r2_score(dataset['train'][target], y_pred_train)}")

    print(f"- Test Set")
    print(f"\tMean Squared Error: {mean_squared_error(dataset['test'][target], y_pred_test)}")
    print(f"\tMedian Absolute Error: {median_absolute_error(dataset['test'][target], y_pred_test)}")
    print(f"\tr-Squared: {r2_score(dataset['test'][target], y_pred_test)}")

## Linear Regression
Implemented with ElasticNet, which combines L1 and L2 regularization

In [9]:
linear_model_params = {
    'alpha':[10, 100, 1000, 10000],
    'l1_ratio':[0.40, 0.60]
}

linear_model = GridSearchCV(estimator=ElasticNet(), param_grid=linear_model_params, 
                            scoring=['neg_mean_absolute_error', 'r2'], 
                            refit='r2', cv=4)

In [8]:
fit_test_print(linear_model, 'Linear Model L1/L2 Regularized', 'incomes', 'Income')

16:14:08 - Training Linear Model L1/L2 Regularized
16:26:53 - Done

Results for Income with Linear Model L1/L2 Regularized
- Best parameters: {'alpha': 10, 'l1_ratio': 0.6}
- Training Set
	Mean Squared Error: 3274596194.857993
	Median Absolute Error: 22929.83070571476
	r-Squared: 0.11891055597779776
- Test Set
	Mean Squared Error: 3281850782.70632
	Median Absolute Error: 22924.74442348032
	r-Squared: 0.11853319389596961


In [10]:
fit_test_print(linear_model, 'Linear Model L1/L2 Regularized', 'wages', 'Wage')

16:29:04 - Training Linear Model L1/L2 Regularized
16:37:42 - Done

Results for Wage with Linear Model L1/L2 Regularized
- Best parameters: {'alpha': 10, 'l1_ratio': 0.6}
- Training Set
	Mean Squared Error: 2351956604.698796
	Median Absolute Error: 17558.869250567495
	r-Squared: 0.15201561556297183
- Test Set
	Mean Squared Error: 2333282440.602135
	Median Absolute Error: 17562.433428652887
	r-Squared: 0.15276740461628124


## SGD Regression
Loss set to Huber to be more robust to outliers in income and wage

In [7]:
sgd_model_params = {
    'alpha':[0.0001, 0.001, 0.01]
}

sgd_model = GridSearchCV(estimator=SGDRegressor(penalty='elasticnet', learning_rate='adaptive'), param_grid=sgd_model_params, cv=4)

In [8]:
fit_test_print(sgd_model, 'SGD Model L1/L2 Regularized', 'incomes', 'Income')

22:43:06 - Training SGD Model L1/L2 Regularized
23:35:27 - Done

Results for Income with SGD Model L1/L2 Regularized
- Best parameters: {'alpha': 0.0001}
- Training Set
	Mean Squared Error: 2313730116.8329206
	Median Absolute Error: 12513.364387014095
	r-Squared: 0.3787176688793734
- Test Set
	Mean Squared Error: 2288257245.7425137
	Median Absolute Error: 12522.970271169928
	r-Squared: 0.380348437367074


In [9]:
income_pred = pd.Series(sgd_model.predict(dataset['test']['features']))
df = pd.concat([dataset['test']['incomes'].reset_index(drop=True), income_pred], axis=1)
print(df)
print(df.corr())


PINCP             0
0       12000.0  26969.389940
1           0.0  -4974.186043
2           0.0  -9717.373729
3       40000.0  77796.252504
4       13000.0  57601.649867
...         ...           ...
642903      0.0  12223.399332
642904      0.0   1882.580636
642905  44000.0  65213.996765
642906      0.0  -1778.753408
642907      0.0 -18568.921707

[642908 rows x 2 columns]
          PINCP         0
PINCP  1.000000  0.616728
0      0.616728  1.000000


In [10]:
fit_test_print(sgd_model, 'Logistic Model L1/L2 Regularized', 'wages', 'Wage')

23:35:28 - Training Logistic Model L1/L2 Regularized
00:21:39 - Done

Results for Wage with Logistic Model L1/L2 Regularized
- Best parameters: {'alpha': 0.0001}
- Training Set
	Mean Squared Error: 1565194628.2093012
	Median Absolute Error: 9285.146097658673
	r-Squared: 0.4363179455367291
- Test Set
	Mean Squared Error: 1536207636.0796754
	Median Absolute Error: 9283.257766323872
	r-Squared: 0.4396291032552606


## Keras Neural Net Regression
Implemented with feed-forward dense layers 

In [72]:
type(list(dataset['train']['incomes']))

list

In [13]:
# a function to initialize the model
def init_keras():
    model = Sequential()
    model.add(Dense(200, input_dim=151))
    model.add(ELU())
    model.add(Dense(200))
    model.add(LeakyReLU())
    model.add(Dense(50))
    model.add(LeakyReLU())
    model.add(Dense(20))
    model.add(LeakyReLU())
    model.add(Dense(1))
    model.compile(loss='huber_loss', optimizer='adam', metrics=['mean_absolute_error', 'cosine_proximity'])
    return model

neural_net_model = KerasRegressor(build_fn=init_keras, epochs=50, batch_size=32, verbose=1)

In [12]:
fit_test_print(neural_net_model, 'Keras Neural Net', 'incomes', 'Income', grid=0)

00:28:59 - Training Keras Neural Net
Instructions for updating:
Colocations handled automatically by placer.


ValueError: Please provide as model targets either a single array or a list of arrays. You passed: y=<PandasArray>
[21300.0,     0.0, 67600.0,     0.0,     0.0, 25000.0, 82000.0, 17000.0,
 30000.0,     0.0,
 ...
     0.0, 53000.0,     0.0,     0.0,     0.0,  1300.0,  9800.0, 55000.0,
     0.0,     0.0]
Length: 2571631, dtype: float64

In [ ]:
fit_test_print(neural_net_model, 'Keras Neural Net', 'wages', 'Wage', grid=0)